In [2]:
import numpy as np
import pandas as pd

import pgeocode
from geopy.geocoders import GoogleV3

In [ ]:
nomi = pgeocode.Nominatim('se')
#print(nomi.query_location(['Stockholm, sweden']))
postal_codes = df_full.query('`PostNr (Leverans)` != "0"')['PostNr (Leverans)'].astype(str).unique().tolist()
postal_codes = [x[:3] + ' '+x[3:] for x in postal_codes]
postal_code_df = nomi.query_postal_code(postal_codes)
postal_code_df.sort_values(by= "postal_code", inplace = True)
postal_code_df

In [3]:
def get_address(df, join_df):
    '''Function to create address column concatenating other address text columns'''
    
    df['postal_code_new']= [''.join(x.split(' ')) for x in df['postal_code']]
    df = pd.merge(df, join_df[["PostNr (Leverans)", "Län", "Kommun"]].drop_duplicates(), how= "left", left_on = "postal_code_new", right_on= "PostNr (Leverans)")
    df['Address'] = df['postal_code'] + ', ' + df['Kommun'] + ', '+  df['Län'] + ', '+ "Sweden"
    
    return df

In [ ]:
API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
geolocator = GoogleV3(api_key=API_KEY)

missing_address = postal_code_df.query('latitude.isnull()')
missing_address = get_address(missing_address, df_full)
# Apply geolocator.geocode to the address column
missing_address["loc"]= missing_address["Address"].apply(geolocator.geocode)
# create longitude, laatitude and altitude from location column (returns tuple) from the geocode’s response, if it’s not None.
missing_address["point"] = missing_address["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
# Split the column "point" into separate columns 'lat' 'lon' and 'altitude'
missing_address[['lat', 'lon', 'altitude']] = pd.DataFrame(missing_address['point'].to_list(), index=missing_address.index)

In [ ]:
missing_address_subset = missing_address[["postal_code","lat", "lon"]]
missing_address_subset.columns = ["postal_code", "latitude", "longitude"]
missing_address_subset

In [ ]:
# concat the missing list and postal code df from OpenStreetMap
final_list = pd.concat([missing_address_subset, 
                        postal_code_df.loc[~ postal_code_df.latitude.isnull(), ["postal_code", "latitude", "longitude"]]])
print(final_list.shape)
print(final_list.info())
final_list.head()

In [ ]:
# To check if there any missing values
final_list[final_list["latitude"].isnull()]

In [ ]:
# Append a new postal code to the existing list
final_list = final_list.append({'postal_code': "573 74", 
               "latitude": 57.82482969839215,
               "longitude": 15.277196956108165}, ignore_index=True)

# save the list as a csv file
final_list.to_csv("lit_of_postal_codes.csv", sep= ",", index=False)

In [ ]:
postal_data = pd.read_csv(f'{PATH}/SE/SE.txt', sep="\t", header=None)
postal_data.columns = ["country_code", "postal_code", "place_name", "admin_name1",
                      "admin_code1", "admin_name2",
                      "admin_code2", "admin_name3",
                      "admin_code3", "latitude", "longitude", "accuracy"]
postal_data

In [ ]:
# import geopandas as gpd
# import geopy


# locator = geopy.Nominatim(user_agent="myGeocoder")
# location = locator.geocode("365 32, LESSEBO, Kronobergs län, Sweden")
# location

# from geopy.extra.rate_limiter import RateLimiter

# # 1 - conveneint function to delay between geocoding calls
# geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# # 2- - create location column
# missing_['location'] = missing_['Address'].apply(geocode)
# # 3 - create longitude, laatitude and altitude from location column (returns tuple)
# missing_['point'] = missing_['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# # 4 - split point column into latitude, longitude and altitude columns
# missing_[['lat', 'long', 'altitude']] = pd.DataFrame(missing_['point'].tolist(), index=missing_.index)